
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [494]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
from scipy import stats


In [495]:
# Choose which results to use
results_to_use = ["Adaptive", "Zero"]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['Adaptive', 'Random']


In [496]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [497]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: Adaptive
                                    0                    1
0                             command                  run
1                       num_of_epochs                10000
2                  num_of_simulations                  100
3                           grid_size                    3
4                      queue_capacity                   10
5                     congestion_rate                 0.07
6                       with_hotspots                 True
7                           wage_time                   10
8                      credit_balance                    5
9           shared_auction_parameters                 None
10  adaptive_auction_action_selection   e_greedy_exp_decay
11               bid_calculation_rule               linear
12       adaptive_auction_update_rule        simple_bandit
13             auction_episode_length                   10
14   action_selection_hyperparameters                 None
15    adaptive_auction_discretization 

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [498]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("the standard deviation of number of trips for the first two simulations are: " + str(np.std(all_average_number_of_trips[0])) + " and " + str(np.std(all_average_number_of_trips[1])))
mean_1 = round(np.mean(all_average_number_of_trips[0]),3)
mean_2 = round(np.mean(all_average_number_of_trips[1]),3)
std_1 = round(np.std(all_average_number_of_trips[0]),3)
std_2 = round(np.std(all_average_number_of_trips[1]),3)
print("in latex:","$(M=",mean_1,", SD=",std_1,")$","$(M=",mean_2,", SD=",std_2,")$")
print("in latex:", "t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")


TtestResult(statistic=6.487702312450497, pvalue=6.827255774506673e-10, df=198.0)
The average number of trips for the first two simulations are: 24888.73 and 24750.39
the standard deviation of number of trips for the first two simulations are: 147.5366974687993 and 152.4698589885883
in latex: $(M= 24888.73 , SD= 147.537 )$ $(M= 24750.39 , SD= 152.47 )$
in latex: t( 198.0 )= 6.488 , p= 0.0
The p-value is: 6.827255774506673e-10
The difference is significant


## 2. Average Congestion of Central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [499]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")
    
mean_1 = round(np.mean(average_congestion_per_intersection[0][:,1,1]),3)
mean_2 = round(np.mean(average_congestion_per_intersection[1][:,1,1]),3)
std_1 = round(np.std(average_congestion_per_intersection[0][:,1,1]),3)
std_2 = round(np.std(average_congestion_per_intersection[1][:,1,1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

TtestResult(statistic=-25.555108396393067, pvalue=1.2987407719253222e-64, df=198.0)
The average congestion for the first two simulations are: 0.30803945 and 0.32891525
The p-value is: 1.2987407719253222e-64
The difference is significant
in latex: $(M= 0.308 ,\ SD= 0.006 )$ $(M= 0.329 ,\ SD= 0.006 )$
in latex: $t( 198.0 )= -25.555 , p= 0.0 $


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [500]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(average_time_agent[0]),3)
mean_2 = round(np.mean(average_time_agent[1]),3)
std_1 = round(np.std(average_time_agent[0]),3)
std_2 = round(np.std(average_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(max_time_agent[0]),3)
mean_2 = round(np.mean(max_time_agent[1]),3)
std_1 = round(np.std(max_time_agent[0]),3)
std_2 = round(np.std(max_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ",\ p=", round(results[1], 3), "$")

TtestResult(statistic=-50.68727275851144, pvalue=2.381544171587639e-115, df=198.0)
The average time waited for the first two simulations are: 0.23983044444444446 and 0.2772946944444445
The p-value is: 2.381544171587639e-115
in latex: $(M= 0.24 ,\ SD= 0.006 )$ $(M= 0.277 ,\ SD= 0.005 )$
in latex: $t( 198.0 )= -50.687 , p= 0.0 $
TtestResult(statistic=-51.231694314253254, pvalue=3.333802695237549e-116, df=198.0)
The max time waited for the first two simulations are: 0.6207537777777777 and 0.715976888888889
The p-value is: 3.333802695237549e-116
in latex: $(M= 0.621 ,\ SD= 0.014 )$ $(M= 0.716 ,\ SD= 0.013 )$
in latex: $t( 198.0 )= -51.232 ,\ p= 0.0 $


In [501]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        # print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        # print(results)
        # print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        # print("The p-value is: " + str(results[1]))
        mean_1 = round(np.mean(average_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(average_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(average_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(average_time_intersection[1][:,i,j]),3)
        print("Intersection [", i, "][", j, "]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")
        # print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
        # print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ",\ p=", round(results[1], 3), "$")
        

Intersection [ 0 ][ 0 ]:  Adaptive $(M= 0.027 ,\ SD= 0.005 )$ , Zero $(M= 0.024 ,\ SD= 0.004 ),\ t( 198 )= 5.81 ,\ p= 0.0 $
Intersection [ 0 ][ 1 ]:  Adaptive $(M= 0.233 ,\ SD= 0.02 )$ , Zero $(M= 0.241 ,\ SD= 0.018 ),\ t( 198 )= -2.68 ,\ p= 0.008 $
Intersection [ 0 ][ 2 ]:  Adaptive $(M= 0.027 ,\ SD= 0.005 )$ , Zero $(M= 0.023 ,\ SD= 0.003 ),\ t( 198 )= 5.76 ,\ p= 0.0 $
Intersection [ 1 ][ 0 ]:  Adaptive $(M= 0.342 ,\ SD= 0.038 )$ , Zero $(M= 0.318 ,\ SD= 0.032 ),\ t( 198 )= 4.71 ,\ p= 0.0 $
Intersection [ 1 ][ 1 ]:  Adaptive $(M= 0.906 ,\ SD= 0.05 )$ , Zero $(M= 1.304 ,\ SD= 0.042 ),\ t( 198 )= -60.8 ,\ p= 0.0 $
Intersection [ 1 ][ 2 ]:  Adaptive $(M= 0.284 ,\ SD= 0.027 )$ , Zero $(M= 0.275 ,\ SD= 0.025 ),\ t( 198 )= 2.45 ,\ p= 0.015 $
Intersection [ 2 ][ 0 ]:  Adaptive $(M= 0.026 ,\ SD= 0.006 )$ , Zero $(M= 0.023 ,\ SD= 0.004 ),\ t( 198 )= 3.66 ,\ p= 0.0 $
Intersection [ 2 ][ 1 ]:  Adaptive $(M= 0.288 ,\ SD= 0.035 )$ , Zero $(M= 0.265 ,\ SD= 0.03 ),\ t( 198 )= 4.92 ,\ p= 0.0 $
Inter

In [502]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        # print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        mean_1 = round(np.mean(max_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(max_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(max_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(max_time_intersection[1][:,i,j]),3)
        print("Intersection [", i, "][", j, "]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")
        # print(results)
        # print("The average time waited for the first two simulations are: " + str(np.mean(max_time_intersection[0][:,i,j])) + " and " + str(np.mean(max_time_intersection[1][:,i,j])))
        # print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]:  Adaptive $(M= 0.095 ,\ SD= 0.014 )$ , Zero $(M= 0.084 ,\ SD= 0.011 ),\ t( 198 )= 6.01 ,\ p= 0.0 $
Intersection [ 0 ][ 1 ]:  Adaptive $(M= 0.651 ,\ SD= 0.049 )$ , Zero $(M= 0.674 ,\ SD= 0.047 ),\ t( 198 )= -3.43 ,\ p= 0.001 $
Intersection [ 0 ][ 2 ]:  Adaptive $(M= 0.093 ,\ SD= 0.014 )$ , Zero $(M= 0.084 ,\ SD= 0.008 ),\ t( 198 )= 5.95 ,\ p= 0.0 $
Intersection [ 1 ][ 0 ]:  Adaptive $(M= 0.914 ,\ SD= 0.095 )$ , Zero $(M= 0.866 ,\ SD= 0.074 ),\ t( 198 )= 3.98 ,\ p= 0.0 $
Intersection [ 1 ][ 1 ]:  Adaptive $(M= 2.09 ,\ SD= 0.122 )$ , Zero $(M= 3.073 ,\ SD= 0.103 ),\ t( 198 )= -61.28 ,\ p= 0.0 $
Intersection [ 1 ][ 2 ]:  Adaptive $(M= 0.776 ,\ SD= 0.066 )$ , Zero $(M= 0.761 ,\ SD= 0.066 ),\ t( 198 )= 1.61 ,\ p= 0.108 $
Intersection [ 2 ][ 0 ]:  Adaptive $(M= 0.09 ,\ SD= 0.016 )$ , Zero $(M= 0.083 ,\ SD= 0.012 ),\ t( 198 )= 3.82 ,\ p= 0.0 $
Intersection [ 2 ][ 1 ]:  Adaptive $(M= 0.788 ,\ SD= 0.088 )$ , Zero $(M= 0.737 ,\ SD= 0.074 ),\ t( 198 )= 4.41 ,\ p= 0.0 $
Int

In [503]:
###### Grid based######

# Average time
print("AVERAGE TIME WAITED")
average_time_grid = []
for exp in results_to_use:
    average_time_grid.append(
        all_sim_results[exp]['stat_average_time_waited_grid'])

means_1 = []
for i in range(len(average_time_grid[0])):
    means_1.append(np.mean(average_time_grid[0][i]))
means_2 = []
for i in range(len(average_time_grid[1])):
    means_2.append(np.mean(average_time_grid[1][i]))

print(np.mean(average_time_grid[0], axis=0))
# print("mean = ", np.mean(np.mean(average_time_grid[0], axis=0)))
# print("std = ", np.std(np.mean(average_time_grid[0], axis=0)))

results = stats.ttest_ind(means_1, means_2)
print(results)
print("The average time waited for the first two simulations are: " +
      str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(means_1), 3)
mean_2 = round(np.mean(means_2), 3)
std_1 = round(np.std(means_1), 3)
std_2 = round(np.std(means_2), 3)
print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
      "$(M=", mean_2, ",\ SD=", std_2, ")$")
print("in latex:", "$t(", results.df, ")=", round(
    results[0], 3), ", p=", round(results[1], 3), "$")










# # Max time
# print("MAX TIME WAITED")
# max_time_grid = []
# for exp in results_to_use:
#     max_time_grid.append(
#         all_sim_results[exp]['stat_max_time_waited_grid'])

# means_1 = []
# for i in range(len(max_time_grid[0])):
#     means_1.append(np.mean(max_time_grid[0][i]))
# means_2 = []
# for i in range(len(max_time_grid[1])):
#     means_2.append(np.mean(max_time_grid[1][i]))


# results = stats.ttest_ind(means_1, means_2)
# print(results)
# print("The max time waited for the first two simulations are: " +
#       str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
# print("The p-value is: " + str(results[1]))
# mean_1 = round(np.mean(means_1), 3)
# mean_2 = round(np.mean(means_2), 3)
# std_1 = round(np.std(means_1), 3)
# std_2 = round(np.std(means_2), 3)
# print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
#       "$(M=", mean_2, ",\ SD=", std_2, ")$")
# print("in latex:", "$t(", results.df, ")=", round(
#     results[0], 3), ", p=", round(results[1], 3), "$")

AVERAGE TIME WAITED
[[0.0271565  0.233166   0.0268695 ]
 [0.34171025 0.906314   0.28369175]
 [0.02603975 0.288191   0.02533525]]
TtestResult(statistic=-50.687272758511455, pvalue=2.381544171587504e-115, df=198.0)
The average time waited for the first two simulations are: 0.23983044444444446 and 0.2772946944444445
The p-value is: 2.381544171587504e-115
in latex: $(M= 0.24 ,\ SD= 0.006 )$ $(M= 0.277 ,\ SD= 0.005 )$
in latex: $t( 198.0 )= -50.687 , p= 0.0 $


## 4. Gini coefficient <a id="4"></a>

In [504]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
# Present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_time_waited[0]),3)
mean_2 = round(np.mean(all_ginis_time_waited[1]),3)
std_1 = round(np.std(all_ginis_time_waited[0]),3)
std_2 = round(np.std(all_ginis_time_waited[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=23.61012818528581, pvalue=1.7638171254215093e-59, df=198.0)
The average gini of time waited for the first two simulations are: 0.14131020591026344 and 0.1342498075588954
The p-value is: 1.7638171254215093e-59
in latex: $(M= 0.141 ,\ SD= 0.002 )$ $(M= 0.134 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 23.61 , p= 0.0 $


In [505]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_satisfaction[0]),3)
mean_2 = round(np.mean(all_ginis_satisfaction[1]),3)
std_1 = round(np.std(all_ginis_satisfaction[0]),3)
std_2 = round(np.std(all_ginis_satisfaction[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=23.61012818528581, pvalue=1.7638171254215093e-59, df=198.0)
The average gini of satisfaction for the first two simulations are: 0.4254760946572833 and 0.43398041667481313
The p-value is: 1.7638171254215093e-59
in latex: $(M= 0.425 ,\ SD= 0.002 )$ $(M= 0.434 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 23.61 , p= 0.0 $


## 5. Trip Satisfaction <a id="5"></a>

In [506]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfaction for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_satisfactions[0]),3)
mean_2 = round(np.mean(all_satisfactions[1]),3)
std_1 = round(np.std(all_satisfactions[0]),3)
std_2 = round(np.std(all_satisfactions[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")


TtestResult(statistic=10.285614780730773, pvalue=3.738318338738298e-20, df=198.0)
The average satisfaction for the first two simulations are: 0.15633658081673904 and 0.15501660368408524
The p-value is: 3.738318338738298e-20
in latex: $(M= 0.156 ,\ SD= 0.001 )$ $(M= 0.155 ,\ SD= 0.001 )$
in latex: $t( 198.0 )= 10.286 , p= 0.0 $


## 6. Auction Reward <a id="5"></a>

In [507]:
######Intersection based - Average Auction Reward######

average_reward_intersection = []
for exp in results_to_use:
    average_reward_intersection.append(
        all_sim_results[exp]['stat_average_auction_reward_per_intersection'])
    
for i in range(len(average_reward_intersection[0][0])):
    for j in range(len(average_reward_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_reward_intersection[0][:,i,j], average_reward_intersection[1][:,i,j])
        print(results)
        print("The average auction reward for the first two simulations are: " + str(np.mean(average_reward_intersection[0][:,i,j])) + " and " + str(np.mean(average_reward_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=5.37253599830821, pvalue=2.1676664679131907e-07, df=198.0)
The average auction reward for the first two simulations are: 0.05166941666666667 and 0.04907166666666665
The p-value is: 2.1676664679131907e-07
Intersection [ 0 ][ 1 ]
TtestResult(statistic=5.775212064275728, pvalue=2.9388103477041827e-08, df=198.0)
The average auction reward for the first two simulations are: 0.25268666666666667 and 0.24559133333333327
The p-value is: 2.9388103477041827e-08
Intersection [ 0 ][ 2 ]
TtestResult(statistic=4.555596728391792, pvalue=9.122244116795075e-06, df=198.0)
The average auction reward for the first two simulations are: 0.05181191666666665 and 0.04981458333333332
The p-value is: 9.122244116795075e-06
Intersection [ 1 ][ 0 ]
TtestResult(statistic=9.05159747443914, pvalue=1.3381396899524995e-16, df=198.0)
The average auction reward for the first two simulations are: 0.2633661666666666 and 0.2517284166666666
The p-value is: 1.3381396899524995e-16
In